# 패키지들은 연습 문제를 풀며 연습하기 위해 별도 로드 X

# Question 1.
- 서비스 이탈예측 데이터.
- 2 유형 환경과 일치시키기 위해 y_train과 x_train을 하나의 train 데이터 셋으로 병합하여 진행.

## 1. 패키지들 로드

In [1]:
import pandas as pd

## 1. 데이터 병합 및 확인 단계

In [23]:
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv')

train = pd.concat([x_train, y_train['Exited']], axis = 1)
display(train.info())
display(x_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       6499 non-null   int64  
 1   Surname          6499 non-null   object 
 2   CreditScore      6499 non-null   int64  
 3   Geography        6499 non-null   object 
 4   Gender           6499 non-null   object 
 5   Age              6499 non-null   int64  
 6   Tenure           6499 non-null   int64  
 7   Balance          6499 non-null   float64
 8   NumOfProducts    6499 non-null   int64  
 9   HasCrCard        6499 non-null   int64  
 10  IsActiveMember   6499 non-null   int64  
 11  EstimatedSalary  6499 non-null   float64
 12  Exited           6499 non-null   int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 660.2+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3501 entries, 0 to 3500
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       3501 non-null   int64  
 1   Surname          3501 non-null   object 
 2   CreditScore      3501 non-null   int64  
 3   Geography        3501 non-null   object 
 4   Gender           3501 non-null   object 
 5   Age              3501 non-null   int64  
 6   Tenure           3501 non-null   int64  
 7   Balance          3501 non-null   float64
 8   NumOfProducts    3501 non-null   int64  
 9   HasCrCard        3501 non-null   int64  
 10  IsActiveMember   3501 non-null   int64  
 11  EstimatedSalary  3501 non-null   float64
dtypes: float64(2), int64(7), object(3)
memory usage: 328.3+ KB


None

In [ ]:
# 1. 데이터의 통계량 확인
display(train.describe(include = ['object', 'category']))
display(train.describe())

# 결과 변수
#   - count: 결측치를 제외한 전체 데이터의 수 (행의 수와 일치).
#   - unique: 서로 다른 범주의 수 (범주 종류의 수).
#   - top: 최빈 범주
#   - freq: 최빈 범주의 개수.

,Surname,Geography,Gender
count,6499,6499,6499
unique,2289,3,4
top,Brown,France,Male
freq,21,3227,3485


,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,6.499000e+03,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000
mean,1.569157e+07,650.396830,38.957070,5.041545,76836.581068,1.519772,0.708878,0.514387,100346.564524,0.203724
std,7.187584e+04,96.618957,10.502803,2.891779,62407.570894,0.578975,0.454314,0.499831,57944.655305,0.402797
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,1.562949e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,50907.565000,0.000000
50%,1.569181e+07,651.000000,37.000000,5.000000,97560.160000,1.000000,1.000000,1.000000,100496.840000,0.000000
75%,1.575358e+07,718.000000,44.000000,8.000000,127844.690000,2.000000,1.000000,1.000000,150480.155000,0.000000
max,1.581566e+07,850.000000,92.000000,10.000000,238387.560000,4.000000,1.000000,1.000000,199970.740000,1.000000


## 변수 선택 단계

In [49]:
# Surname과 CustomerId는 제외
#   - Surname: 범주의 종류가 2289개로 의미없는 정보임.
#   - CustomerId: Id 컬럼이기에 제외
train_df = train.drop(columns = ['Surname', 'CustomerId'])
x_test_df = x_test.drop(columns = ['Surname', 'CustomerId'])

## Encoding 단계

In [58]:
# 범주형 변수만 따로 추출
target = train.drop(columns = 'Exited')
cat_lst = target.select_dtypes(exclude = 'number').columns.tolist()
cat_lst

# 원-핫 인코딩 적용
#   - drop_first = False
#       - False: 기준 범주 없이 모든 범주를 0과 1로 encoding 할 때 사용한다. 이는 기준 범주 없이 개별 범주로 처리하기 때문.
#       - True: 통계모델에서 기준 범주를 설정할 때 사용한다. 통계에서는 기준 범주를 기준으로하여 변화량을 측정하기 때문. (이 경우 기준범주가 0이 된다).
train_encoded = pd.get_dummies(train, columns = cat_lst, drop_first = False)
x_test_encoded = pd.get_dummies(x_test, columns = cat_lst, drop_first = False)

In [ ]:
display(train_encoded)

In [14]:
# 1. 결과변수가 무엇인지 확인
temp = train.columns[~train.columns.isin(test.columns)]
print(temp)

Index(['Exited'], dtype='object')
